In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.append('/dfs/scratch0/vschen/metal')

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Generate Data

In [4]:
seed = 312

import random
np.random.seed(seed)
random.seed(seed)

In [5]:
from simulate import data_config
from synthetics_utils import generate_synthetic_data
from visualization_utils import visualize_data, display_scores, plot_slice_scores
# X, Y, C, L = generate_synthetic_data(data_config, verbose=True)

# # L[L[:, 2] != 0, 2] = 0 # remove LF2 to show data underneath
# visualize_data(X, Y, C, L)
# data_config

In [ ]:
config = {'N': 10000,
 'mus': np.array([[-3.5,  0], [5, 0]]),
 'labels': [-1, 1],
 'props': [0.10, 0.90],
 'variances': [3, 5],
 'accs': np.array([0.99, 0.99]),
 'covs': [('recall', 0.99),
  ('recall', 0.99)],
  'head_config': None,
  'mv_normal': False
}

def generate_simple_data(config, x_var=None, x_range=None, verbose=False):
    X, Y, C, L = generate_synthetic_data(config, verbose=True)

    # LF1
    L = np.zeros((config['N'], 2))
    lf_num = 0
    slice_0_label = -1
    r = 3 #HACK
    h, k = (-3.5, 0)

    # shifts
    h+=1

    lf_idx = np.sqrt((X[:, 0] - h) ** 2 + (X[:, 1] - k) ** 2) < r
    L[lf_idx, lf_num] = slice_0_label
    
    # LF1
    lf_num = 1
    slice_1_label = 1
    r = 7 #HACK
    h, k = (5, 0)

    # shifts
    h-=1

    lf_idx = np.sqrt((X[:, 0] - h) ** 2 + (X[:, 1] - k) ** 2) < r
    L[lf_idx, lf_num] = slice_1_label
    
    return X, Y, C, L

X, Y, C, L = generate_simple_data(config)
visualize_data(X, Y, C, L)

In [ ]:
from simulate import simulate, experiment_config
split_idx = int(len(X) * experiment_config["train_prop"])
X_train, X_test = X[:split_idx], X[split_idx:]
Y_train, Y_test = Y[:split_idx], Y[split_idx:]  # no gt train data!
L_train, L_test = L[:split_idx], L[split_idx:]
C_train, C_test = C[:split_idx], C[split_idx:]

accs = []
from metal.metrics import accuracy_score
for lf_idx in range(L_train.shape[1]):
    voted_idx = L_test[:, lf_idx] != 0
    accs.append(accuracy_score(L_test[voted_idx, lf_idx], Y_test[voted_idx]))

print (accs)

In [ ]:
Y_tilde

In [ ]:
from metal.contrib.slicing.experiment_utils import generate_weak_labels
Y_tilde = generate_weak_labels(L_train, np.array(accs))
plt.hist(Y_tilde[:, 0], 10)

In [ ]:
from metal.end_model import EndModel
from metal.contrib.slicing.online_dp import MLPModule, SliceDPModel

model_configs = {
    "EndModel": {
        "base_model_class" : EndModel,
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "base_model_init_kwargs": {
            "layer_out_dims": [10, 2],
            "input_layer_config": {
                "input_relu": False,
                "input_batchnorm": False,
                "input_dropout": 0.0,
            }
        },
        "train_on_L": False
    },
    "AttentionModel": {
        "base_model_class" : SliceDPModel,
        "base_model_init_kwargs": {
            "reweight": True,
            "r": 10,
            "slice_weight": 0.5,
            "L_weights": None
        },
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "train_on_L": True,
    }
}

In [ ]:
config['accs'] = np.array(accs)

In [ ]:
from simulate import simulate, experiment_config

experiment_config['use_weak_labels_from_gen_model'] = False
experiment_config['x_var'] = None
experiment_config['num_trials'] = 1
# experiment_config['x_range'] = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
experiment_config['verbose'] = True
experiment_config['visualize_data'] = True
experiment_config['train_kwargs']['print_every'] = 1
experiment_config['train_kwargs']['l2'] = 0
experiment_config['train_kwargs']['lr'] = 0.005
experiment_config['train_kwargs']['n_epochs'] = 30
experiment_config['seed'] = 444
scores = \
    simulate(config, generate_simple_data, experiment_config, model_configs)
# display_scores(scores, experiment_config['x_var'], [None])
scores

In [ ]:
from simulate import data_config as config

config['accs'] = np.array([0.99, 0.99, 0.99, 0.99])
config['covs'] = [('recall', 0.99), ('recall', 0.99), ('recall', 0.99), ('recall', 0.99)]
config['head_config']['r'] = 1.2
X, Y, C, L = generate_synthetic_data(config, verbose=True)

# L[L[:, 2] != 0, 2] = 0 # remove LF2 to show data underneath
visualize_data(X, Y, C, L)
config

## Overlapping slices of different classes

In [ ]:
from simulate import data_config as config

config = {
  'N': 10000,
 'mus': [np.array([-5,  0]), np.array([3, 0])],
 'labels': [1, 1],
 'props': [0.05, 0.95],
 'variances': [1, 5],
 'head_config': {'h': -1, 'k': 0, 'r': 3, 'slice_label': -1},
 'accs': np.array([0.99, 0.99, 0.99]),
 'covs': [('recall', 0.99), ('recall', 0.99), ('precision', 0.75)]
}
X, Y, C, L = generate_synthetic_data(config, verbose=True)

# L[L[:, 2] != 0, 2] = 0 # remove LF2 to show data underneath
visualize_data(X, Y, C, L)
config

In [ ]:
from metal.end_model import EndModel
from metal.contrib.slicing.online_dp import MLPModule, SliceDPModel

model_configs = {
    "EndModel": {
        "base_model_class" : EndModel,
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "base_model_init_kwargs": {
            "layer_out_dims": [10, 2],
            "input_layer_config": {
                "input_relu": False,
                "input_batchnorm": False,
                "input_dropout": 0.0,
            }
        },
        "train_on_L": False
    },
    "UniformModel": {
        "base_model_class" : SliceDPModel,
        "base_model_init_kwargs": {
            "reweight": False,
            "r": 10,
            "slice_weight": 0.1,
            "L_weights": np.array([1., 1., 1.]).astype(np.float32)
        },
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "train_on_L": True
    },
    "ManualModel": {
        "base_model_class" : SliceDPModel,
        "base_model_init_kwargs": {
            "reweight": False,
            "r": 10,
            "slice_weight": 0.1,
            "L_weights": np.array([1., 1., 5.]).astype(np.float32) # LF2 w/ 5x weight
        },
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "train_on_L": True
    },
    "AttentionModel": {
        "base_model_class" : SliceDPModel,
        "base_model_init_kwargs": {
            "reweight": True,
            "r": 10,
            "slice_weight": 0.1,
            "L_weights": None
        },
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "train_on_L": True,
    }
}

In [ ]:
from simulate import simulate, experiment_config

experiment_config['use_weak_labels_from_gen_model'] = False
experiment_config['x_var'] = None
experiment_config['num_trials'] = 1
# experiment_config['x_range'] = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
experiment_config['verbose'] = True
experiment_config['visualize_data'] = True
experiment_config['train_kwargs']['print_every'] = 1
experiment_config['train_kwargs']['l2'] = 1e-4
experiment_config['train_kwargs']['lr'] = 0.005
experiment_config['train_kwargs']['n_epochs'] = 30
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
experiment_config['seed'] = 123
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
experiment_config['seed'] = 432
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

## Slices in same class that are overlooked

In [ ]:
from simulate import data_config as config

config['accs'] = np.array([0.99, 0.99, 0.99, 0.99])
config['covs'] = [('recall', 0.99), ('recall', 0.99), ('recall', 0.99), ('recall', 0.99)]
config['head_config']['r'] = 1.2
X, Y, C, L = generate_synthetic_data(config, verbose=True)

# L[L[:, 2] != 0, 2] = 0 # remove LF2 to show data underneath
visualize_data(X, Y, C, L)
config

In [ ]:
from metal.end_model import EndModel
from metal.contrib.slicing.online_dp import MLPModule, SliceDPModel

model_configs = {
#     "EndModel": {
#         "base_model_class" : EndModel,
#         "input_module_class": MLPModule,
#         "input_module_init_kwargs": {
#             'input_dim': 2,
#             'middle_dims': [10, 10],
#             'bias': True,
#             'output_dim': 10
#          },
#         "base_model_init_kwargs": {
#             "layer_out_dims": [10, 2],
#             "input_layer_config": {
#                 "input_relu": False,
#                 "input_batchnorm": False,
#                 "input_dropout": 0.0,
#             }
#         },
#         "train_on_L": False
#     },
#     "UniformModel": {
#         "base_model_class" : SliceDPModel,
#         "base_model_init_kwargs": {
#             "reweight": False,
#             "r": 10,
#             "slice_weight": 0.5,
#             "L_weights": np.array([1., 1., 1., 1.]).astype(np.float32)
#         },
#         "input_module_class": MLPModule,
#         "input_module_init_kwargs": {
#             'input_dim': 2,
#             'middle_dims': [10, 10],
#             'bias': True,
#             'output_dim': 10
#          },
#         "train_on_L": True
#     },
#     "ManualModel": {
#         "base_model_class" : SliceDPModel,
#         "base_model_init_kwargs": {
#             "reweight": False,
#             "r": 10,
#             "slice_weight": 0.5,
#             "L_weights": np.array([1., 1., 5., 1.]).astype(np.float32) # LF2 w/ 5x weight
#         },
#         "input_module_class": MLPModule,
#         "input_module_init_kwargs": {
#             'input_dim': 2,
#             'middle_dims': [10, 10],
#             'bias': True,
#             'output_dim': 10
#          },
#         "train_on_L": True
#     },
    "AttentionModel": {
        "base_model_class" : SliceDPModel,
        "base_model_init_kwargs": {
            "reweight": True,
            "r": 10,
            "slice_weight": 0.5,
            "L_weights": None
        },
        "input_module_class": MLPModule,
        "input_module_init_kwargs": {
            'input_dim': 2,
            'middle_dims': [10, 10],
            'bias': True,
            'output_dim': 10
         },
        "train_on_L": True,
    }
}

In [ ]:
from simulate import simulate, experiment_config
experiment_config['use_weak_labels_from_gen_model'] = False
experiment_config['x_var'] = None
experiment_config['num_trials'] = 1
# experiment_config['x_range'] = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
experiment_config['verbose'] = True
experiment_config['visualize_data'] = True
experiment_config['train_kwargs']['print_every'] = 1
# experiment_config['train_kwargs']['l2'] = 1e-3
experiment_config['train_kwargs']['l2'] = 0
experiment_config['train_kwargs']['lr'] = 0.005
experiment_config['train_kwargs']['n_epochs'] = 50
experiment_config['train_kwargs']['scheduler_config'] = {
            "verbose": True,
            "scheduler": "exponential",
            # ['constant', 'exponential', 'reduce_on_plateau']
            # Freeze learning rate initially this many epochs
            "lr_freeze": 0,
            # Scheduler - exponential
            "exponential_config": {"gamma": 0.9},  # decay rate
            # Scheduler - reduce_on_plateau
            "plateau_config": {
                "factor": 0.1,
                "patience": 0,
                "threshold": 0.0001,
                "min_lr": 1e-7,
                "verbose": True,
                "mode":'max'
            },
        }

experiment_config['seed'] = 321
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

experiment_config['seed'] = 111
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
experiment_config['seed'] = 432
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
experiment_config['seed'] = 1515
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
experiment_config['seed'] = 33
scores = \
    simulate(config, generate_synthetic_data, experiment_config, model_configs)
display_scores(scores, experiment_config['x_var'], [None])

In [ ]:
# def eval_on_slices(model, X_test, Y_test, C_test):
#     S0_idx, S1_idx, S2_idx = (
#         np.where(C_test == 0)[0],
#         np.where(C_test == 1)[0],
#         np.where(C_test == 2)[0],
#     )
#     eval_dict = {"S0": S0_idx, "S1": S1_idx, "S2": S2_idx}

#     preds, Y = model._get_predictions((X_test, Y_test), return_probs=False)
    
#     print ("S0:", np.sum((preds == Y)[S0_idx]) / len(Y[S0_idx]))
#     print ("S1:", np.sum((preds == Y)[S1_idx]) / len(Y[S1_idx]))
#     print ("S2:", np.sum((preds == Y)[S2_idx]) / len(Y[S2_idx]))

In [ ]:
# accs = data_config['accs']

# split_idx = int(len(X) * 0.8)

# X = torch.Tensor(X)
# # X = X.astype(np.float32)
# Y_cat = Y.copy().astype(np.int32)
# Y_cat[Y==-1] = 2
# X_train, X_test = X[:split_idx], X[split_idx:]
# Y_train, Y_test = Y_cat[:split_idx], Y_cat[split_idx:]
# L_train, L_test = L[:split_idx], L[split_idx:]
# C_train, C_test = C[:split_idx], C[split_idx:]

# from metal.contrib.slicing.experiment_utils import generate_weak_labels
# Y_tilde = generate_weak_labels(L_train, accs)

In [ ]:
# # from simulate import simulate, data_config, experiment_config, model_configs
# from simulate import simulate, data_config, experiment_config

# experiment_config['use_weak_labels_from_gen_model'] = False
# experiment_config['visualize_data'] = True
# experiment_config['seed'] = seed
# experiment_config['x_var'] = 'sp'
# experiment_config['x_range'] = [0.1]
# experiment_config['num_trials'] = 2
# experiment_config['train_kwargs']['print_every'] = 1
# experiment_config['verbose'] = True
# sp_scores = \
#     simulate(data_config, generate_synthetic_data, experiment_config, model_configs)
# display_scores(sp_scores, experiment_config['x_var'], experiment_config['x_range'])

## Vary Slice Porportion 
_Ratio fo Green to Orange in Top Right figure_

In [9]:
from simulate import simulate, data_config, experiment_config, model_configs
experiment_config['use_weak_labels_from_gen_model'] = True
experiment_config['x_var'] = 'sp'
experiment_config['num_trials'] = 1
# experiment_config['x_range'] = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
experiment_config['x_range'] = np.linspace(0.05, 0.2, 3)
experiment_config['verbose'] = False
experiment_config['seed'] = False
sp_scores = \
    simulate(data_config, generate_synthetic_data, experiment_config, model_configs)
display_scores(sp_scores, experiment_config['x_var'], experiment_config['x_range'])

Simulating: sp=0.05


Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels

Simulating: sp=0.125


Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels

Simulating: sp=0.2


Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels
Hardcoding 2 -> cat labels



,EndModel,UniformModel,ManualModel,AttentionModel
sp: 0.05,,,,
S0,0.970986,0.970986,0.969052,0.970986
S1,0.944961,0.947105,0.944961,0.947105
overall,0.956375,0.959125,0.956625,0.959125


,EndModel,UniformModel,ManualModel,AttentionModel
sp: 0.125,,,,
S0,0.901354,0.901354,0.901354,0.901354
S1,0.900236,0.901021,0.901021,0.901021
overall,0.898125,0.899375,0.899375,0.899375


,EndModel,UniformModel,ManualModel,AttentionModel
sp: 0.2,,,,
S0,0.845261,0.733075,0.851064,0.851064
S1,0.837031,0.709898,0.837884,0.837884
overall,0.838500,0.718250,0.841000,0.841000


In [ ]:
plot_slice_scores(sp_scores, xlabel="Slice Proportion")

In [ ]:
sp_scores

## Vary Head LF Accuracy
_blue to orange dots in bottom right figure_

In [ ]:
%%time

from simulate import simulate, experiment_config, model_configs
data_config['head_config']['r'] = 2
experiment_config['use_weak_labels_from_gen_model'] = True
experiment_config['x_var'] = 'acc'
experiment_config['num_trials'] = 10
experiment_config['x_range'] = [0.8, 0.85, 0.9, 0.95]
lf_acc_scores = \
    simulate(data_config, generate_synthetic_data, experiment_config, model_configs)

display_scores(lf_acc_scores, experiment_config['x_var'], experiment_config['x_range'])

In [ ]:
plot_slice_scores(lf_acc_scores, xlabel="Head Accuracy", 
                  custom_xranges={'S2': [0.8, 0.85, 0.9, 0.95]})

## Vary Head LF Precision
_num of blue dots in red slice over num points in slice in bottom right_

In [ ]:
%%time

from simulate import simulate, experiment_config, model_configs

data_config['head_config']['r'] = 2
experiment_config['use_weak_labels_from_gen_model'] = True
experiment_config['x_var'] = 'cov.precision'
experiment_config['num_trials'] = 10
experiment_config['x_range'] = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
lf_prec_scores = \
    simulate(data_config, generate_synthetic_data, experiment_config, model_configs)

display_scores(lf_prec_scores, experiment_config['x_var'], experiment_config['x_range'])

In [ ]:
plot_slice_scores(lf_prec_scores, xlabel="Head LF Precision")
#                   custom_ylims={"S2":[0, 0.2]})

## Vary Head LF Recall
_num blue dots in red slice over num red dots in bottom right_

In [ ]:
%%time

from simulate import simulate, data_config, experiment_config, model_configs

data_config['head_config']['r'] = 2
experiment_config['use_weak_labels_from_gen_model'] = True
experiment_config['x_var'] = 'cov.recall'
experiment_config['num_trials'] = 10
# experiment_config['x_range'] = [0.95]
# experiment_config['verbose'] = True
experiment_config['x_range'] = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


lf_rec_scores = simulate(data_config, generate_synthetic_data, experiment_config, model_configs)
display_scores(lf_rec_scores, experiment_config['x_var'], experiment_config['x_range'])

In [ ]:
plot_slice_scores(lf_rec_scores, xlabel="Head LF Recall")